In [1]:
import os
from math import log
import sys
#import snap
import numpy as np
import queue
from scipy.misc import logsumexp
import math
from scipy.misc import comb


class Node(object):
    def __init__(self, id, inEdges, outEdges):
        self.id = id
        self.inEdges = inEdges
        self.outEdges = outEdges

class Edge(object):
    def __init__(self, id, node1, node2, probabilities):
        self.id = id
        self.node1 = node1
        self.node2 = node2
        self.probabilities = probabilities
        self.estimates = np.random.rand(len(self.probabilities))

class Graph(object):
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = edges

##Where are topic probabilities

class Item(object):
    def __init__(self, id, topicDistribution):
        self.id = id
        self.topicDistribution = topicDistribution
        self.topicEstimates = np.zeros(len(self.topicDistribution))

class CascadeLog(object):
    def __init__(self, node1, node2, edge, time):
        self.node1 = node1
        self.node2 = node2
        self.edge = edge
        self.time = time

def generateGraph(nodes, topics, density=0.5):
    graph = Graph([],[])
    for node in range(nodes):
        graph.nodes.append(Node(node, {}, {}))

    edgeProbs = np.random.rand(nodes,nodes)
    edgeOccurence = edgeProbs < density
    edgeIndices = np.transpose(edgeOccurence.nonzero())
    for id,edge in enumerate(edgeIndices):
        node1 = edge[0]
        node2 = edge[1]
        probabilities = np.random.rand(topics)
        edge = Edge(id,node1,node2,probabilities)
        graph.edges.append(edge)
        graph.nodes[node1].outEdges[id] = node2
        graph.nodes[node2].inEdges[id] = node1
    return graph

class TIC(object):
    def __init__(self, graph, numTopics, items):
        self.graph = graph
        self.numTopics = numTopics
        self.numItems = len(items)
        self.items = items
        

    def diffusion(self, seeds, item, isEnvironment, returnCascade=False):
        possibleWorld = self.generatePossibleWorld()
        influencedNodes = seeds[:]
        activeNodes = queue.Queue()
        influencedMap = {x:0 for x in range(len(self.graph.nodes))}
        edgeCascade = {x:0 for x in range(len(self.graph.edges))}
        for seed in seeds:
            activeNodes.put([seed,0])
            influencedMap[seed] = 1
        while not activeNodes.empty():
            currentNode, timestep = activeNodes.get()
            for edge, node in self.graph.nodes[currentNode].outEdges.items():
                if influencedMap[node] == 0:
                    if isEnvironment:
                        dot = np.dot(self.graph.edges[edge].probabilities, item.topicDistribution)
                    else:
                        dot = np.dot(self.graph.edges[edge].estimates, item.topicEstimates)                
                    if dot > possibleWorld[edge]:
                        activeNodes.put([node, timestep+1])
                        influencedNodes.append(node)
                        influencedMap[node] = 1
                        edgeCascade[edge] = 1
        if returnCascade:
            return len(influencedNodes), edgeCascade
        else:
            return len(influencedNodes)
        
         
        
    def expectedSpread(self, seeds, item, numberOfSimulations, isEnvironment):
        expSpread = 0.0
        for simulation in range(numberOfSimulations):
            expSpread += self.diffusion(seeds, item, isEnvironment)
        expSpread = expSpread/numberOfSimulations
        return expSpread

    def findBestSeeds(self, item, budget, isEnvironment=True, numberOfSimulations=100):
        seeds = []
        seedMap = {}
        while len(seeds) < budget:
            maximumSpread = 0
            newSeed = -1
            for candidate in range(len(self.graph.nodes)):
                try:
                    isSeed = seedMap[candidate]
                except:
                    expSpread = self.expectedSpread(seeds+[candidate], item, numberOfSimulations, isEnvironment)
                    if expSpread > maximumSpread:
                        maximumSpread = expSpread
                        newSeed = candidate
            if newSeed != -1:
                seeds.append(newSeed)
                seedMap[newSeed] = 1
            else:
                break
        return seeds, maximumSpread

    def generatePossibleWorld(self):
        coinFlips = np.random.rand(len(self.graph.edges))
        return coinFlips
    
    def timGreedy(self,rrSets,item, budget, isEnvironment):
        seeds = []
        seedsCovered = set()
        rrSetsCovered = set()
        while len(seeds) < budget:
            s = 1
            print("Choosing seed : ",s)
            s += 1
            maxCoverage = 0
            maxCandidate = -1
            maxCoverageList = [] 
            newSeed = -1
            for candidate in range(len(self.graph.nodes)):
                coverage=0
                coverageList = []
                if candidate in seedsCovered:
                    continue
                for rrSet in rrSets:
                    if rrSet in rrSetsCovered:
                        continue
                    if candidate in rrSets[rrSet]:
                        coverage +=1
                        coverageList.append(rrSet)
                if coverage > maxCoverage:
                    maxCoverage = coverage
                    maxCandidate = candidate
                    maxCoverageList = coverageList
            print("Adding seed: ",maxCandidate)
            print("Coverage",maxCoverage)
            seeds.append(maxCandidate)
            seedsCovered.add(maxCandidate)
            rrSetsCovered.update(maxCoverageList)
        return seeds

                      
    def getRRSetWidth(self,rrSet):
        width = 0
        for nodeId in rrSet:
            width += len(self.graph.nodes[nodeId].inEdges)
        return width
    
    def generateRRSet(self,item, isEnvironment,debug=False):
        #print("toGenerate rrset")
        coinFlips = self.generatePossibleWorld()
        liveEdges = set()
        if (debug):
            print("total edges: ",len(self.graph.edges))
        edges = 0
        for e in range(len(self.graph.edges)):
            if isEnvironment:
                dot = np.dot(self.graph.edges[e].probabilities, item.topicDistribution)
            else:
                dot = np.dot(self.graph.edges[e].estimates, item.topicEstimates)
            if coinFlips[e] >= dot:
                edges += 1
                liveEdges.add(e)
        if(debug):
            print("sampled edges",edges)
        sampledNode = np.random.randint(0,len(self.graph.nodes)) # assuming ids start from 0
        if(debug):
            print("Sampled node",sampledNode)
        bfsQueue = queue.Queue()
        bfsQueue.put(sampledNode)
        processedNodes = set()
        processedNodes.add(sampledNode)
        while not bfsQueue.empty():
            qNode = bfsQueue.get()
            edges = self.graph.nodes[qNode].inEdges
            for edge in edges:
                if edge not in liveEdges:
                    continue
                if self.graph.edges[edge].node2 != qNode:
                    print("bfs Error/n")
                nNode = self.graph.edges[edge].node1
                if nNode not in processedNodes: 
                    bfsQueue.put(nNode)
                    processedNodes.add(nNode)
        if(debug):
            print("size of rrset: ",len(processedNodes))
            print("rrset: ",processedNodes)
        #print("Generated rrset")
        return processedNodes
    
    def kptEstimation(self,item,budget,isEnvironment,l):
        KPT = 1
        n = len(self.graph.nodes)
        m = len(self.graph.edges)
        rangeV = round(log(n,2) - 1)
        for i in range(rangeV):
            ci = round((6*l*math.log(n,2)+6*log(log(n,2)))*(2**i))
            sumK = 0
            for j in range(ci):
                sampledSet = self.generateRRSet(item,isEnvironment)
                w = self.getRRSetWidth(sampledSet)
                kappa = 1 - (1 - w/m)**budget
                sumK += kappa
            if sumK/ci > 1/(2**i) :
                KPT = n * sumK/(2 * ci)
                print("KPT",KPT)
                return KPT
        print("KPT",KPT)
        return KPT
                
        
    
    def getRRSetEstimate(self,item, budget, isEnvironment):
        #Values of l and e decide accuracy and time complexity
        l = 0.7
        KPT = self.kptEstimation(item, budget, isEnvironment,l)
        n = len(self.graph.nodes)
        e = 0.1
        k=budget
        lamda = (8+2*e)*n*(l*log(n) + log(comb(n,k)) + log(2))*(e**-2)
        estimate = round(lamda/KPT)
        print("estimate",estimate)
        return estimate,KPT
        
        
    
    
    def tim(self, item, budget, isEnvironment=True):
        rrSetCount,KPT = self.getRRSetEstimate(item, budget, isEnvironment)
        rrSets={}
        # range
        for i in range(rrSetCount):
            if i < 3:
                debug = True
            else:
                debug = False
            rrSets[i] = self.generateRRSet(item, isEnvironment,debug)
        seeds = self.timGreedy(rrSets,item, budget, isEnvironment)
        return seeds,KPT


class MAB(object):
    def __init__(self, tic, budget):
        self.budget = budget
        self.tic = tic

    
    def explore(self, item):
        seeds = np.random.choice(len(self.tic.graph.nodes), self.budget, replace=False)
        return seeds.tolist()

    def exploit(self, item):
        seeds,_ = self.tic.findBestSeeds(item, self.budget, isEnvironment=False)
        return seeds
    
    def exploit_tim(self, item):
        seeds,_ = self.tic.tim(item, self.budget, isEnvironment=False)
        return seeds

    def epsilonGreedy(self, epsilon, item):
        if epsilon > np.random.rand(1):
            seeds = self.explore(item)
        else:
            seeds = self.exploit(item)
            #seeds = self.exploit_tim(item)
        spread, cascade = self.tic.diffusion(seeds=seeds, item=item, isEnvironment=True, returnCascade=True)
        return cascade

    def tryTim(self,iterations):
        print("starting tim")
        print(self.budget)
        for it in range(iterations):
            print("Iter: ",it)
            for i in range(self.tic.numItems):
                seeds, spread = self.tic.tim(self.tic.items[i], self.budget, isEnvironment=True)
                print("seeds : ",seeds)
                print("spread lower bound: ",spread)
                #self.tic.diffusion(seeds, self.tic.items[i], isEnvironment=True, returnCascade=False)
                expSpread = self.tic.expectedSpread(seeds, self.tic.items[i], numberOfSimulations=100, isEnvironment=True)
                print("expected spread:", expSpread)
                dseeds, maxSpread = self.tic.findBestSeeds(self.tic.items[i], self.budget, isEnvironment=True, numberOfSimulations=100)
                print("Diffusion seeds:", dseeds)
                print("Max Spread: ", maxSpread)
                

    
    def learner(self, iterations, epsilon):
        pi = np.random.rand(numTopics)
        pi = np.log(pi/sum(pi))
        edgeActivation = {x:{i:0.0 for i in range(self.tic.numItems)} for x in range(len(self.tic.graph.edges))}
        #print("cp1")
        for t in range(iterations):
            for i in range(self.tic.numItems):
                cascade = self.epsilonGreedy(epsilon, self.tic.items[i])
                cascadeLogProbPositive = np.zeros(self.tic.numTopics)
                cascadeLogProbNegative = np.zeros(self.tic.numTopics)
                for edge, isActive in cascade.items():
                    if isActive == 1:
                        cascadeLogProbPositive += np.log(self.tic.graph.edges[edge].estimates)
                        edgeActivation[edge][i] += 1
                    else:
                        cascadeLogProbNegative += np.log(1. - self.tic.graph.edges[edge].estimates)
                self.tic.items[i].topicEstimates = (pi + cascadeLogProbPositive + cascadeLogProbNegative)
                self.tic.items[i].topicEstimates = np.exp(self.tic.items[i].topicEstimates - logsumexp(self.tic.items[i].topicEstimates))
            pi = np.sum([self.tic.items[i].topicEstimates for i in range(self.tic.numItems)],0)/self.tic.numItems
            normalizer = pi[:]*self.tic.numItems
            pi = np.log(pi)
            for edge in range(len(self.tic.graph.edges)):
                self.tic.graph.edges[i].estimates = np.sum([(self.tic.items[i].topicEstimates * edgeActivation[edge][i])/(t+1) for i in range(self.tic.numItems)],0)/normalizer

itemList = []
numItems = 1
numTopics = 1
for i in range(numItems):
    itemDist = np.random.rand(numTopics)
    #itemList.append(Item(i, itemDist/sum(itemDist)))
    print(itemDist/sum(itemDist))
    itemList.append(Item(i, itemDist/sum(itemDist)))
tic = TIC(generateGraph(100, numTopics, density=0.04), numTopics, itemList)
mab = MAB(tic, 3)
mab.tryTim(1)





[ 1.]
starting tim
3
Iter:  0
KPT 32.7997338985353
estimate 39776
total edges:  366
sampled edges 166
Sampled node 80
size of rrset:  4
rrset:  {80, 58, 35, 52}
total edges:  366
sampled edges 181
Sampled node 18
size of rrset:  76
rrset:  {0, 2, 5, 6, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 26, 27, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 67, 68, 70, 72, 73, 75, 76, 77, 78, 80, 82, 83, 84, 85, 86, 89, 90, 92, 93, 94, 95, 97, 99}
total edges:  366
sampled edges 197
Sampled node 20
size of rrset:  66
rrset:  {0, 1, 2, 6, 7, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 26, 27, 29, 30, 31, 32, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 50, 52, 54, 55, 56, 57, 60, 61, 62, 65, 66, 68, 70, 71, 72, 76, 79, 81, 82, 83, 84, 86, 89, 90, 91, 92, 93, 94, 95, 97, 99}
Choosing seed :  1
Adding seed:  32
Coverage 30217
Choosing seed :  1
Adding seed:  58
Coverage 1216
Choosing seed :  1
A